In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import matplotlib
import seaborn as sns

In [9]:
df = pd.read_csv('./data/assessments.csv')
df

,code_module,code_presentation,id_assessment,assessment_type,date,weight
0,AAA,2013J,1752,TMA,19.0,10.0
1,AAA,2013J,1753,TMA,54.0,20.0
2,AAA,2013J,1754,TMA,117.0,20.0
3,AAA,2013J,1755,TMA,166.0,20.0
4,AAA,2013J,1756,TMA,215.0,30.0
...,...,...,...,...,...,...
201,GGG,2014J,37443,CMA,229.0,0.0
202,GGG,2014J,37435,TMA,61.0,0.0
203,GGG,2014J,37436,TMA,124.0,0.0
204,GGG,2014J,37437,TMA,173.0,0.0


# 이탈 예측 
<br>

### 우선순위 1: 가장 중요한 컬럼 (이탈 예측의 핵심)
| 컬럼명          | 설명                                   | 이유                                                   |
|---------------|--------------------------------------|------------------------------------------------------|
| date|강의 시작 후 평가 일정 (일 단위) | 특정 시점에서 학습자가 이탈하는 패턴 분석 가능 |
| weight | 평가의 성적 반영 비율 (%) | 중요한 시험(TMA, Exam) 이후 이탈 증가 가능성 있음 |
| assessment_type | 평가 유형 (TMA, CMA, Exam) | 시험 유형에 따라 학습자의 반응이 다를 수 있음 |

- 분석 방향
    - 날짜(date)를 기준으로 특정 시험(weight 높은 평가) 이후 학습자가 감소하는지 분석
    - 특정 평가 유형(assessment_type)이 이탈에 미치는 영향 분석

<br><br>

### 우선순위 2: 중간 중요도 (강좌별 차이 분석)
| 컬럼명             | 설명                          | 이유                                   |
|------------------|---------------------------|--------------------------------------|
| code_module | 강좌 코드 (AAA, GGG 등) | 강좌별 이탈률이 다를 가능성이 높음 |
| code_presentation | 강좌 제공 학기 (2013J, 2014J 등) | 특정 학기별 이탈률 차이 분석 가능 |

- 분석 방향
    - 특정 강좌나 특정 학기(code_presentation)에서 이탈률이 유독 높은지 확인
    - 강좌별 학습 난이도 차이가 영향을 미칠 가능성 고려

<br><br>

### 우선순위 3: 보조 데이터 (추가적인 패턴 분석용)
| 컬럼명             | 설명                          | 이유                                   |
|------------------|---------------------------|--------------------------------------|
| id_assessment     | 평가 고유 ID     | 특정 평가에서 유독 많은 이탈이 발생하는지 확인 가능      |

- 분석 방향
    - 특정 시험(id_assessment)에서 유독 이탈률이 높은 경우 확인

In [12]:
df

,code_module,code_presentation,id_assessment,assessment_type,date,weight
0,AAA,2013J,1752,TMA,19.0,10.0
1,AAA,2013J,1753,TMA,54.0,20.0
2,AAA,2013J,1754,TMA,117.0,20.0
3,AAA,2013J,1755,TMA,166.0,20.0
4,AAA,2013J,1756,TMA,215.0,30.0
...,...,...,...,...,...,...
201,GGG,2014J,37443,CMA,229.0,0.0
202,GGG,2014J,37435,TMA,61.0,0.0
203,GGG,2014J,37436,TMA,124.0,0.0
204,GGG,2014J,37437,TMA,173.0,0.0


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206 entries, 0 to 205
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   code_module        206 non-null    object 
 1   code_presentation  206 non-null    object 
 2   id_assessment      206 non-null    int64  
 3   assessment_type    206 non-null    object 
 4   date               195 non-null    float64
 5   weight             206 non-null    float64
dtypes: float64(2), int64(1), object(3)
memory usage: 9.8+ KB


In [11]:
df.describe()

,id_assessment,date,weight
count,206.000000,195.000000,206.000000
mean,26473.975728,145.005128,20.873786
std,10098.625521,76.001119,30.384224
min,1752.000000,12.000000,0.000000
25%,15023.250000,71.000000,0.000000
50%,25364.500000,152.000000,12.500000
75%,34891.750000,222.000000,24.250000
max,40088.000000,261.000000,100.000000
